<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/ICaRL---Lucia/ICaRLMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [0]:
import os
import logging
import sys

In [2]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone -b ICaRL---Lucia https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 806 (delta 50), reused 6 (delta 2), pack-reused 719
Receiving objects: 100% (806/806), 552.26 KiB | 1.61 MiB/s, done.
Resolving deltas: 100% (502/502), done.


# Import packages

In [0]:

from DatasetCIFAR.data_set import Dataset 
from DatasetCIFAR import ResNet
from DatasetCIFAR import utils
from DatasetCIFAR import params
from DatasetCIFAR import ICaRLModel
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np

from torchvision import transforms
from torch.utils.data import Subset, DataLoader
from torch.nn import functional as F
import random
random.seed(params.SEED)

# Define Datasets

In [0]:
train_transformer = transforms.Compose([transforms.RandomCrop(size = 32, padding=4),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transformer = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
trainDS = Dataset(train=True, transform = train_transformer)
testDS = Dataset(train=False, transform = test_transformer)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [6]:
train_splits = trainDS.splits
test_splits = testDS.splits
print(train_splits)
print(test_splits)

[[94.0, 63.0, 74.0, 21.0, 35.0, 56.0, 91.0, 96.0, 87.0, 48.0], [68.0, 80.0, 22.0, 37.0, 60.0, 97.0, 51.0, 62.0, 92.0, 76.0], [75.0, 89.0, 23.0, 99.0, 39.0, 66.0, 54.0, 69.0, 84.0, 61.0], [85.0, 24.0, 98.0, 41.0, 73.0, 58.0, 78.0, 77.0, 70.0, 49.0], [65.0, 88.0, 36.0, 93.0, 45.0, 10.0, 90.0, 17.0, 32.0, 59.0], [83.0, 43.0, 53.0, 11.0, 86.0, 19.0, 38.0, 30.0, 40.0, 50.0], [57.0, 81.0, 12.0, 95.0, 25.0, 47.0, 34.0, 52.0, 44.0, 72.0], [46.0, 79.0, 20.0, 28.0, 5.0, 71.0, 8.0, 18.0, 33.0, 15.0], [55.0, 29.0, 64.0, 31.0, 67.0, 7.0, 13.0, 14.0, 42.0, 6.0], [82.0, 2.0, 27.0, 16.0, 26.0, 3.0, 4.0, 1.0, 9.0, 0.0]]
[[94.0, 63.0, 74.0, 21.0, 35.0, 56.0, 91.0, 96.0, 87.0, 48.0], [68.0, 80.0, 22.0, 37.0, 60.0, 97.0, 51.0, 62.0, 92.0, 76.0], [75.0, 89.0, 23.0, 99.0, 39.0, 66.0, 54.0, 69.0, 84.0, 61.0], [85.0, 24.0, 98.0, 41.0, 73.0, 58.0, 78.0, 77.0, 70.0, 49.0], [65.0, 88.0, 36.0, 93.0, 45.0, 10.0, 90.0, 17.0, 32.0, 59.0], [83.0, 43.0, 53.0, 11.0, 86.0, 19.0, 38.0, 30.0, 40.0, 50.0], [57.0, 81.0, 12.

# Define Network

In [0]:
ICaRL = ICaRLModel.ICaRLStruct(dataset = trainDS)

# Train phase

In [0]:
pars_tasks = []
test_indexes = []

for task in range(0, 100, params.TASK_SIZE):
  pars_tasks.insert(task, 0)

In [0]:
for task in range(0, 100, params.TASK_SIZE):

  train_indexes = trainDS.__getIndexesGroups__(task)
  test_indexes = test_indexes + testDS.__getIndexesGroups__(task)

  train_dataset = Subset(trainDS, train_indexes)
  test_dataset = Subset(testDS, test_indexes)

  train_loader = DataLoader( train_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE)
  test_loader = DataLoader( test_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE )

  ICaRL.updateRep(task, train_dataset, train_splits[int(task/10)], train_transformer)

  m = ICaRL.k / (task + params.TASK_SIZE) #task + params.TASK_SIZE represents the total amount of classes know at each step
  print('m: ', m)
  ICaRL.reduceExemplars(m)

  for y in train_splits[int(task/10)]:
      images = []
      for i, el in enumerate(train_dataset):
        if( el[1] == y):
          images.append(train_indexes[i])
      ICaRL.generateExemplars(images, int(m), int(y) )
      total = 0.0
      correct = 0.0
      for img, lbl, _ in train_loader:
          img = img.float().to(params.DEVICE)
          preds = ICaRL.classify(img, train_splits[: int(task/10) + 1])
          total += lbl.size(0)
          correct += (preds.data.cpu() == lbl).sum()
      print('Train Accuracy: %d %%' % (100 * correct / total))

In [0]:
total = 0.0
correct = 0.0
for img, lbl, _ in train_loader:
    img = img.float().to(params.DEVICE)
    preds = ICaRL.classify(img, train_splits[: int(task/10) + 1])
    total += lbl.size(0)
    correct += (preds.data.cpu() == lbl).sum()

In [0]:
def classify(self, x, col):
    '''
    x -> [BATCH SIZE] images to be classified
    col -> list classes see until now

    '''
    examplars = self.exemplar
    phi = self.features_extractor

    features = []
    for P_y in col:
      P_y = int(P_y)
      #itero per tutte le classi in analisi
      #in P_y io ho m elementi, ovvero gli exemplars per quella specifica classe
      if(self.exemplar[P_y] is not None):
      for ex in self.exemplar[P_y]:
          #exemplar contine gli indici delle immagini di riferiemnto
          ex = dataSet._data[ex]
          feature = phi(ex) 
          feature = feature.squeeze()
          feature.data /= feature.data.norm()
          features.append(feature)
        features = torch.stack(features)
        mu_y = features.mean(0).squeeze()
        mu_y.data = mu_y.data / mu_y.data.norm() # Normalize
        exemplar_means.append(mu_y)
    self.exemplar_means = exemplar_means

    means = torch.stack(exemplar_means) # (n_classes, feature_size)
    means = torch.stack([means] * batch_size) # (batch_size, n_classes, feature_size)
    means = means.transpose(1, 2) # (batch_size, feature_size, n_classes)

    feature = self.features_extractor(x) # (batch_size, feature_size)
    for i in xrange(feature.size(0)): # Normalize
      feature.data[i] = feature.data[i] / feature.data[i].norm()
    feature = feature.unsqueeze(2) # (batch_size, feature_size, 1)
    feature = feature.expand_as(means) # (batch_size, feature_size, n_classes)

    dists = (feature - means).pow(2).sum(1).squeeze() #(batch_size, n_classes)
    _, preds = dists.min(1)

    return preds
